## Setup
Import and util functions

In [1]:
import numpy as np

from components.Accelerator import *
from components.ColumnAccumulator import *
from components.OperationalArray import *
from components.RowAccumulator import *
from components.WordAccumulator import *

In [2]:
def convert_matrix_to_oa_shape(matrix, max_rows=128, max_cols=128, bit_depth=8):
    """
    Convert a 2D matrix of 8-bit unsigned integers (shape (r, c)) into
    a shape (max_rows, max_cols * bit_depth) suitable for the OperationalArray.
    The first r rows get the expanded bits of the matrix's c columns.
    The rest are zeros.

    We'll place matrix[i,j]'s bits into columns [j*bit_depth : j*bit_depth+bit_depth].
    """
    r, c = matrix.shape
    # Prepare an empty big array
    big_weights = np.zeros((max_rows, max_cols * bit_depth), dtype=int)

    for i in range(r):
        for j in range(c):
            val = matrix[i, j]
            # Convert to 8-bit
            for b in range(bit_depth):
                # For bit b: if b=0 => LSB, b=7 => MSB
                bit_mask = (val >> (bit_depth - 1 - b)) & 1
                big_weights[i, j * bit_depth + b] = bit_mask
    return big_weights


In [7]:
def log_vector_and_matrix(vector, matrix, file_prefix, vector_bit_precision=8, matrix_bit_precision=8, rows=None):
    """
    Logs:
      - vector in decimal
      - vector in binary (bit_precision-defined), padded to 'rows' if specified
      - matrix in decimal
      - matrix in bit-unrolled form (128x(128 * matrix_bit_precision))

    Args:
        vector (np.ndarray): 1D array of unsigned integers.
        matrix (np.ndarray): 2D array of unsigned integers.
        file_prefix (str): Prefix for the output filenames.
        vector_bit_precision (int): Number of bits per vector element (e.g., 4 or 8).
        matrix_bit_precision (int): Number of bits per matrix element (e.g., 8).
        rows (int, optional): Total number of rows to log for the vector. Pads with zeros if greater than vector.size.
    """
    vector_size = vector.size
    total_rows = rows if rows is not None else vector_size

    # 1) Log vector (decimal)
    with open(f"{file_prefix}_vector_decimal.txt", "w") as f:
        for val in vector:
            f.write(str(val) + "\n")
        for _ in range(total_rows - vector_size):
            f.write("0\n")

    # 2) Log vector (binary, bit_precision-wide)
    with open(f"{file_prefix}_vector_bits.txt", "w") as f:
        for val in vector:
            bits = format(val, f'0{vector_bit_precision}b')  # zero-padded binary
            f.write(' '.join(bits) + "\n")
        for _ in range(total_rows - vector_size):
            f.write(' '.join('0' * vector_bit_precision) + "\n")

    # 3) Log matrix (decimal)
    with open(f"{file_prefix}_matrix_decimal.txt", "w") as f:
        for row in matrix:
            f.write(" ".join(str(v) for v in row) + "\n")

    # 4) Log matrix (bit-unrolled) for OperationalArray
    weights_oa = convert_matrix_to_oa_shape(matrix, 128, 128, bit_depth=matrix_bit_precision)
    with open(f"{file_prefix}_matrix_bits.txt", "w") as f:
        for row in weights_oa:
            f.write(' '.join(str(bit) for bit in row) + "\n")


In [4]:
def run_example(vector, matrix):
    """
    1. Convert 'matrix' into 128x(128*8) bit placement.
    2. Create Accelerator(128, 128, 8, 8).
    3. Load weights.
    4. Do the bit-serial multiplication with 'vector'.
    5. Compare with numpy result.
    """
    # Convert matrix to the shape needed by OperationalArray
    weights_oa = convert_matrix_to_oa_shape(matrix, max_rows=128, max_cols=128, bit_depth=8)

    # Create the accelerator
    acc = Accelerator(rows=128, cols=128, bit_depth=8, bit_serial=8)

    # Load weights
    acc.load_weights(weights_oa)

    # Compute hardware-like result
    hw_result = acc.vector_matrix_multiply_bitserial(vector)

    # For clarity, the output length = number of columns in the original matrix
    # because each group of 8 bits in the OA columns maps to 1 "column" in normal math
    # So let's slice it properly:
    out_len = matrix.shape[1]
    final_hw_result = hw_result[:out_len].astype(np.int64)

    # Compare with standard numpy (all 8-bit -> treat as standard unsigned, but safe up to 16-bit or more)
    # We'll do standard 32- or 64-bit integer arithmetic in numpy
    vector_np = vector.astype(np.int64)
    matrix_np = matrix.astype(np.int64)
    np_result = vector_np.dot(matrix_np)  # shape (out_len,)

    print("Vector:", vector)
    print("Matrix shape:", matrix.shape)
    print("Hardware-like result:", final_hw_result)
    print("Numpy result:         ", np_result)
    print("Match?                ", np.allclose(final_hw_result, np_result))
    print("-----\n")

In [5]:
# Example A: Very small, 4x4
# Vector shape: 4
# Matrix shape: (4,4)
# All unsigned 8-bit
np.random.seed(0)

## 8-Bit Examples

### Example 1

In [5]:
vec_A = np.random.randint(0, 256, size=(4,), dtype=np.uint8)
mat_A = np.random.randint(0, 256, size=(4, 4), dtype=np.uint8)

run_example(vec_A, mat_A)

Vector: [172  10 127 140]
Matrix shape: (4, 4)
Hardware-like result: [43018 81868 49043 76794]
Numpy result:          [43018 81868 49043 76794]
Match?                 True
-----



### Example 2

In [8]:
# Example B: 4 x 16
vec_B = np.random.randint(0, 256, size=(4,), dtype=np.uint8)
mat_B = np.random.randint(0, 256, size=(4, 16), dtype=np.uint8)

run_example(vec_B, mat_B)


Vector: [251  82 162 219]
Matrix shape: (4, 16)
Hardware-like result: [ 74709  58463  59802 106817 122956  93995 102995  91238  69837 108493
  96468 120382  91316 109894 128231 102652]
Numpy result:          [ 74709  58463  59802 106817 122956  93995 102995  91238  69837 108493
  96468 120382  91316 109894 128231 102652]
Match?                 True
-----



### Example 3

In [9]:
# Example C: 16 x 16
vec_C = np.random.randint(0, 256, size=(16,), dtype=np.uint8)
mat_C = np.random.randint(0, 256, size=(16, 16), dtype=np.uint8)

run_example(vec_C, mat_C)

Vector: [ 87 168 101 135 174 200 223 122  88  94 107 145  81 139 141 100]
Matrix shape: (16, 16)
Hardware-like result: [273705 305226 310831 337177 250038 224387 254815 346757 234884 274427
 248845 221296 300841 208271 265211 223751]
Numpy result:          [273705 305226 310831 337177 250038 224387 254815 346757 234884 274427
 248845 221296 300841 208271 265211 223751]
Match?                 True
-----



### Example 4

In [10]:
# Example D: 20 x 10 (just to show non-square shapes)
vec_D = np.random.randint(0, 256, size=(20,), dtype=np.uint8)
mat_D = np.random.randint(0, 256, size=(20, 10), dtype=np.uint8)

run_example(vec_D, mat_D)

Vector: [119 236 174 171  11 187 192  43 174 161 219  53  82 220 175  91  91  80
   1  33]
Matrix shape: (20, 10)
Hardware-like result: [244913 346328 320550 296086 329649 282235 373784 235660 264597 380690]
Numpy result:          [244913 346328 320550 296086 329649 282235 373784 235660 264597 380690]
Match?                 True
-----



## Mixed Precision Examples

In [8]:
vec_4x4 = np.random.randint(0, 16, size=(4,), dtype=np.uint8)   # 4-bit vector
mat_4x4 = np.random.randint(0, 256, size=(4, 4), dtype=np.uint8) # 8-bit matrix
log_vector_and_matrix(vec_4x4, mat_4x4,file_prefix="out", vector_bit_precision=4, matrix_bit_precision=8,rows=128)
run_example(vec_4x4, mat_4x4)

Vector: [11  2  2 11]
Matrix shape: (4, 4)
Hardware-like result: [4690 4318 3836 3926]
Numpy result:          [4690 4318 3836 3926]
Match?                 True
-----



In [16]:
vec = np.random.randint(0, 16, size=(64,), dtype=np.uint8)   # 4-bit vector
mat = np.random.randint(0, 256, size=(64, 64), dtype=np.uint8) # 8-bit matrix
run_example(vec, mat)

Vector: [ 3  7  1  7  2 10  5  1 14  7  0  0  3 10 12  1  4 12  7 13  1  0  0  3
  2 10 11  2 11  5  3  5 14 11 14 10  9 12 12  8 10 12  0 11  1 10  1  3
  4 12  6  5 10 11  6 13  6  0  8  7 11 12 13  1]
Matrix shape: (64, 64)
Hardware-like result: [58806 60388 65782 51628 55167 50188 65553 52997 52788 50628 56684 61499
 64327 57475 60555 55141 55445 59280 47538 44622 51902 57321 49640 54267
 50473 56122 53300 49887 58369 46049 59990 54708 56314 53244 54500 59430
 51067 53156 48696 49332 55118 48273 58148 57934 47593 61374 47859 56930
 56924 62289 51802 55222 49082 58944 56700 58419 50535 50875 57654 63309
 56135 60900 48413 54803]
Numpy result:          [58806 60388 65782 51628 55167 50188 65553 52997 52788 50628 56684 61499
 64327 57475 60555 55141 55445 59280 47538 44622 51902 57321 49640 54267
 50473 56122 53300 49887 58369 46049 59990 54708 56314 53244 54500 59430
 51067 53156 48696 49332 55118 48273 58148 57934 47593 61374 47859 56930
 56924 62289 51802 55222 49082 58944 56700 58